In [7]:
import numpy as np
import os
import glob
import pandas as pd
from statsmodels.sandbox.stats.multicomp import multipletests
import numpy as np
from subprocess import call

In [2]:
dir_pre = "/mnt/hpc/home/xuxinran/DirectSeq/data/zhaolin_240206/240201-zhaolin-RNA-merge/v0.7.2/apa/nano_merge"

output_path = f"{dir_pre}_signal_3aqtl.csv"

In [3]:
files = glob.glob(f'{dir_pre}_haplotype_chr*_tmp.csv')
dfs = [pd.read_csv(file) for file in files]
df = pd.concat(dfs, ignore_index=True)
df = df.sort_values(by=['chrom', 'snp_pos_1base'])
df = df.reset_index(drop=True)

In [4]:
df

,chrom,strand,snp_pos_1base,rsID,A1,A2,APA_id_l,MAF,A1_apa1,A2_apa1,A1_apa2,A2_apa2,p_value,beta
0,chr1,+,787399,rs2905055,G,T,"LINC01128.5,LINC01128.3",0.24190,7.0,14.0,1.0,0.0,0.363636,-1.490848
1,chr1,-,879911,rs143853699,T,C,"NOC2L.1,NOC2L.2",0.01442,14.0,7.0,1.0,1.0,1.000000,0.693147
2,chr1,-,935833,rs3128115,G,C,"HES4.2,other",0.11500,10.0,7.0,0.0,0.0,1.000000,0.327213
3,chr1,+,1167796,rs190796582,T,C,"B3GALT6.5,other",0.25250,21.0,77.0,12.0,46.0,1.000000,0.044452
4,chr1,-,1254841,rs10907179,G,C,"CPSF3L.1,other",0.11760,186.0,21.0,45.0,5.0,1.000000,-0.016000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11378,chr9,+,139820421,rs7852970,G,A,"TRAF2.2,TRAF2.1",0.26040,28.0,57.0,0.0,0.0,1.000000,-0.697486
11379,chr9,+,139887042,rs10870159,G,C,"C9orf142.1,C9orf142.2",0.14900,0.0,0.0,10.0,107.0,1.000000,2.304908
11380,chr9,-,139934020,rs6605,T,C,"NPDC1.2,NPDC1.1",0.16670,43.0,94.0,0.0,0.0,1.000000,-0.772750
11381,chr9,-,139934107,rs6604,T,C,"NPDC1.2,NPDC1.1",0.23080,10.0,10.0,0.0,0.0,1.000000,0.000000


In [5]:
p_adjusted = multipletests(df["p_value"], method='fdr_bh')
df['fdr'] = p_adjusted[1]
print("pvalue小于0.05的个数：", len(df[df['p_value']<0.05]))
print("pvalue小于0.01的个数：", len(df[df['p_value']<0.01]))
print("fdr小于0.1的个数：", len(df[df['fdr']<0.1]))

pvalue小于0.05的个数： 570
pvalue小于0.01的个数： 236
fdr小于0.1的个数： 119


In [8]:
result_df = df[df['p_value']<0.05]
result_df.to_csv(output_path, index=False)
call(f'rm {dir_pre}_haplotype_chr*_tmp.csv',shell=True)

0